In [1]:
top_league = [
    'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/', 
    'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/saison_id/',
    'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/saison_id/',
    'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/saison_id/',
    'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/saison_id/',
    'https://www.transfermarkt.com/championship/startseite/wettbewerb/GB2/saison_/id/',
    'https://www.transfermarkt.com/serie-b/startseite/wettbewerb/IT2/saison_id/'
]


In [2]:
import requests
from bs4 import BeautifulSoup


all_team_links = set()  

for i in top_league:
    page = requests.get(i, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, 'html.parser')

    for j in soup.find_all('a', title=True):  
        link = j.get('href') 
        if link and link.startswith("/") and "/kader/" in link:  
            full_link = 'https://www.transfermarkt.com' + link
            all_team_links.add(full_link)


In [3]:
len(all_team_links)

140

In [4]:
all_player_links = []  


for i in all_team_links:
    page = requests.get(i, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, 'html.parser')

    for j in soup.find_all('a', href=True):  
        link = j.get('href')
        if link and "/profil/spieler/" in link:  # Фильтруем только профили игроков
            full_link = 'https://www.transfermarkt.com' + link
            all_player_links.append(full_link)

In [11]:
len(all_player_links)

3822

In [6]:
import pandas as pd

In [9]:
import pandas as pd

df_links = pd.DataFrame(all_player_links, columns=["Player_Link"])

df_links.to_csv("player_links.csv", index=False)



In [12]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup

# Функция для получения рыночной стоимости игрока
def market_value(url):
    try:
        # Заменяем URL для получения данных о стоимости
        url = url.replace("profil", "marktwertverlauf")
        playerid = url.split("spieler/")[1]
        
        # Запрос страницы
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.content, "html.parser")
        
        # Получаем имя игрока
        name_tag = soup.find("h1")
        if not name_tag:
            return None  # Если нет имени, пропускаем

        name = name_tag.get_text(strip=True)

        # Ищем данные о рыночной стоимости в HTML
        script_tags = soup.find_all("script")
        market_value_data = None
        
        for script in script_tags:
            if "Highcharts.Chart" in script.text:
                market_value_data = script.text
                break
        
        if not market_value_data:
            return None  # Если нет данных, пропускаем
        
        # Извлекаем данные с помощью регулярных выражений
        values = re.findall(r'"y":(\d+)', market_value_data)
        clubs = re.findall(r'"verein":"(.*?)"', market_value_data)
        dates = re.findall(r'"datum_mw":"(.*?)"', market_value_data)

        # Преобразуем к числам
        mv = list(map(int, values))
        
        # Преобразуем в DataFrame
        mv_df = pd.DataFrame({"TMId": playerid, "Player": name, "MarketValue": mv, "Club": clubs, "Date": dates})

        # Преобразование даты
        mv_df["Date"] = pd.to_datetime(mv_df["Date"], dayfirst=True, errors="coerce")
        mv_df = mv_df[mv_df["Date"].notnull()]
        
        return mv_df
    
    except Exception as e:
        print(f"Ошибка при обработке {url}: {e}")
        return None



# Сбор данных
all_data = []
for link in all_player_links:
    print(f"Обрабатываю {link}")
    df = market_value(link)
    if df is not None:
        all_data.append(df)
    time.sleep(1)  # Добавляем задержку, чтобы не заблокировали

# Объединяем данные
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_csv("market_values.csv", index=False)
    print("Сбор данных завершен и сохранен в market_values.csv!")
else:
    print("Не удалось собрать данные.")


Обрабатываю https://www.transfermarkt.com/mark-flekken/profil/spieler/125714
Обрабатываю https://www.transfermarkt.com/hakon-rafn-valdimarsson/profil/spieler/488935
Обрабатываю https://www.transfermarkt.com/nathan-collins/profil/spieler/469050
Обрабатываю https://www.transfermarkt.com/sepp-van-den-berg/profil/spieler/541231
Обрабатываю https://www.transfermarkt.com/kristoffer-ajer/profil/spieler/328658
Обрабатываю https://www.transfermarkt.com/ethan-pinnock/profil/spieler/442248
Обрабатываю https://www.transfermarkt.com/ben-mee/profil/spieler/74810
Обрабатываю https://www.transfermarkt.com/ji-soo-kim/profil/spieler/709767
Обрабатываю https://www.transfermarkt.com/rico-henry/profil/spieler/339556
Обрабатываю https://www.transfermarkt.com/aaron-hickey/profil/spieler/591949
Обрабатываю https://www.transfermarkt.com/michael-kayode/profil/spieler/823486
Обрабатываю https://www.transfermarkt.com/vitaly-janelt/profil/spieler/270171
Обрабатываю https://www.transfermarkt.com/christian-norgaard/

In [14]:
all_data

[]